In [3]:
import numpy as np
import json
import os

def generate_nodes(N, space_size=100, energy_node = 100, filename="nodes.json"):
    
    #Sinh n node cảm biến ngẫu nhiên trong không gian 3D và lưu vào folder input_data
    
    np.random.seed(0)  # để kết quả lặp lại
    node_positions = np.random.rand(N, 3) * space_size

    data = []
    for i in range(N):
        data.append({
            "id": i,
            "x": float(node_positions[i][0]),
            "y": float(node_positions[i][1]),
            "z": float(node_positions[i][2]),
            "energy_node": float(energy_node)
        })

    
    os.makedirs("input_data", exist_ok=True)
    filepath = f"input_data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Tạo file {filepath} chứa {N} node.")

# Sinh dữ liệu
for N in [20, 100, 200, 500, 1000]:
    generate_nodes(N, space_size=100, filename=f"nodes_{N}.json")


Tạo file input_data/nodes_20.json chứa 20 node.
Tạo file input_data/nodes_100.json chứa 100 node.
Tạo file input_data/nodes_200.json chứa 200 node.
Tạo file input_data/nodes_500.json chứa 500 node.
Tạo file input_data/nodes_1000.json chứa 1000 node.


In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

input_folder = "input_data"
output_folder = "output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        energies = {d["id"]: d["energy_node"] for d in data}
        # Phân cụm
        clusters_raw = cluster_split(node_positions, node_ids, R=5)
        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c, energies, alpha=0.8)
            #Tính năng lượng trung bình của cụm
            average_energy = round(np.mean([energies[nid] for nid in c["node_ids"]]), 2)
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch),
                "average_energy": average_energy
            }
        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")
        # Vẽ và lưu hình
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab10', len(clusters_output))
        for cid, info in clusters_output.items():
            nodes = np.array([node_positions[nid] for nid in info['nodes']])
            ax.scatter(nodes[:, 0], nodes[:, 1], nodes[:, 2],
                        label=f'Cụm {cid} (E={info["average_energy"]}J)',
                        color=colors(cid))
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                        color=colors(cid),
                        marker='*', s=80, edgecolor='k')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
        plt.title(f'Phân cụm các node cảm biến ({number} node)')
        ax.view_init(elev=30, azim=30)
        plt.tight_layout()
        draw_path = os.path.join(draw_folder, f"nodes_{number}.png")
        plt.savefig(draw_path)
        plt.close(fig)
        print(f"Đã lưu hình vẽ {draw_path}")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\subp

Đã xuất file output_data_kmeans\nodes_100.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ draw_output_kmeans\nodes_100.png
Đã xuất file output_data_kmeans\nodes_1000.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:61: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


...

Đã lưu hình vẽ draw_output_kmeans\nodes_20.png
Đã xuất file output_data_kmeans\nodes_200.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ draw_output_kmeans\nodes_200.png
Đã xuất file output_data_kmeans\nodes_500.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10920\2359092186.py:61: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Đã lưu hình vẽ draw_output_kmeans\nodes_500.png
